In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import GPT2Tokenizer
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 64
batch_size = 128
max_iters = 100
learning_rate = 3e-4
eval_iters = 10
n_embd = 384
n_head=8
n_layer = 8
dropout = 0.3


cuda


In [8]:
# import dataset
dataset = load_dataset("sewon/ambig_qa", "light")

In [9]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [33]:
def encode(text):
    return tokenizer.encode(text, add_special_tokens=True)

def tokenize_dataset(dataset):
    tokenized_dataset = {}
    for split in ['train', 'validation']:
        tokenized_dataset[split] = []
        for example in dataset[split]:
            question = example['question']
            tokenized_question = encode(question)
            tokenized_dataset[split].append(tokenized_question)
    return tokenized_dataset
tokenized_dataset = tokenize_dataset(dataset)

def pad_sequence(seq, max_len, pad_token_id):
    return seq + [pad_token_id] * (max_len - len(seq))


In [34]:
def get_batch(split):
    data_split = tokenized_dataset[split]
    
    if len(data_split) == 0:
        raise ValueError(f"No data available for split: {split}")
    
    indices = random.choices(range(len(data_split)), k=batch_size)
    x = torch.zeros(batch_size, block_size, dtype=torch.long)
    y = torch.zeros(batch_size, block_size, dtype=torch.long)
    for i, idx in enumerate(indices):
        data = data_split[idx]
        if len(data) >= block_size:
            start_idx = random.randint(0, len(data) - block_size - 1)
            x[i] = torch.tensor(data[start_idx:start_idx + block_size], dtype=torch.long)
            y[i] = torch.tensor(data[start_idx + 1:start_idx + block_size + 1], dtype=torch.long)
        else:
            padded_data = pad_sequence(data, block_size + 1, tokenizer.pad_token_id)
            x[i] = torch.tensor(padded_data[:block_size], dtype=torch.long)
            y[i] = torch.tensor(padded_data[1:block_size + 1], dtype=torch.long)
    x, y = x.to(device), y.to(device)
    return x, y

In [21]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'validation']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()    
    return out

In [13]:
vocab_size=len(tokenizer)
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)
        
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            
    def forward(self, index, targets=None):
        B, T = index.shape
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self.forward(index)
            # focus on last time step
            logits = logits[:, -1, :]
            # apply softmax for probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1)
            # append sampled index
            index = torch.cat((index, index_next), dim=1)
        return index




 



In [ ]:
# Instantiate model and optimizer
model = GPTLanguageModel(vocab_size=len(tokenizer))
model = model.to(device)
model_path = 'modelv2.pkl'
model.load_state_dict(torch.load(model_path))
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# Training loop
for iter in range(max_iters):
    if iter % eval_iters == 0:
        try:
            losses = estimate_loss()
            print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['validation']:.4f}")
        except ValueError as e:
            print(e)
            continue

    # Sample a batch of data
    xb, yb = get_batch('train')

    # Evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("Final loss:", loss.item())
# Save the model
torch.save(model.state_dict(),'modelv2.pkl')

step: 0, train loss: 0.5414, val loss: 0.7725
step: 10, train loss: 0.5522, val loss: 0.7588
step: 20, train loss: 0.5476, val loss: 0.7658
step: 30, train loss: 0.5164, val loss: 0.7520
step: 40, train loss: 0.5282, val loss: 0.7731
step: 50, train loss: 0.5077, val loss: 0.7714
